# A/B Testing
- Untuk menguji 2 things anggap saja desain sebuah website apakah desain 1 lebih baik dari desain 2 atau sebaliknya berdasarkan parameter yang sudah ditentukan sebelumnya.

## Story Case
- Rangga! dia dapat tugas baru dari bosnya.
- Kali ini dia disuruh melakukan A/B testing pada website yukbeli.com untuk mengetahui apakah desain website yang baru lebih baik dari desain sebelumnya.
- yukbeli.com merupakan salah satu e-commerce terbaik dan terindah di Indonesia
- Parameter yang rangga gunakan adalah:
    1. Rata-rata waktu yang dihabiskan di website per session
    2. Conversion rate, proporsi dari user yang melakukan transaksi
- Rangga memutuskan untuk melakukan traffic split dengan proporsi desain A sebesar 0.4 dan desain B sebesar 0.6

- Setelah 7 hari melakukan experimen, rangga mendapatkan data seperti di bawah ini dan dia ingin menguji desain mana yang lebih baik menggunakan A/B testing.

## Load Library

In [41]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import chi2

## Load Datasets

In [42]:
data_dict = {
    'number_of_sessions':[6000, 4000],
    'avg_time':[8, 11],
    'stdev_time':[2,3],
    'conversion_rate':[1.5, 2]
}

In [43]:
data = pd.DataFrame(data_dict, index=['A','B'])

In [44]:
data

,number_of_sessions,avg_time,stdev_time,conversion_rate
A,6000,8,2,1.5
B,4000,11,3,2.0


## Dataset Information

In [45]:
data.shape

(2, 4)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, A to B
Data columns (total 4 columns):
number_of_sessions    2 non-null int64
avg_time              2 non-null int64
stdev_time            2 non-null int64
conversion_rate       2 non-null float64
dtypes: float64(1), int64(3)
memory usage: 80.0+ bytes


## Melakukan Uji Statistik
- Rangga akan melakukan 2 uji statistik yaitu:
    1. Z-test untuk parameter avg_time apakah user menghabiskan waktu lebih lama di desain B.
    2. Chi-Square untuk parameter conversion rate apakah desain B memberikan conversion lebih tinggi.

## Z-test
- alpha / confidence level = 0.05

### Hypothesis
- H0 = 2 desain tersebut memiliki avg_time yang sama
- H1 = desain B memiliki avg_time yang lebih tinggi dari desain A

### Calculations

In [47]:
mean_B = 62
mean_A = 60

stdev_B = 45
stdev_A = 40

n_B = 4000
n_A = 6000

Z = (mean_B - mean_A)/np.sqrt(stdev_B**2/n_B + stdev_A**2/n_A)
pvalue = norm.sf(Z)

print(f'Nilai Z adalah {Z}')
print(f'Nilai p-value adalah {pvalue}')

Nilai Z adalah 2.2749070654279993
Nilai p-value adalah 0.011455752709549046


### Conclusion
- Karena nilai p-value < nilai alpha maka reject H0 yang berarti desain B membuat user untuk menghabiskan waktu lebih lama dibandingkan dengan desain A

## Chi-Square
- alpha / confidence level = 0.05

### Hypothesis
- H0 = 2 desain tersebut memiliki conversion rate yang sama
- H1 = desain B memiliki conversion rate yang lebih tinggi dari desain A

### Calculations

In [48]:
#bikin kolom baru yaitu converted dan non-converted
data['converted'] = data['number_of_sessions'] * data['conversion_rate'] / 100
data['non_converted'] = data['number_of_sessions'] - data['converted']
data2 = data[['number_of_sessions', 'converted', 'non_converted']]
data2

,number_of_sessions,converted,non_converted
A,6000,90.0,5910.0
B,4000,80.0,3920.0


- setelah mendapatkan 3 kolom diatas maka langkah selanjutnya adalah menghitung nilai p_hat
- weight = (A_converted + B_converted) / (A_converted + B_converted + A_non_converted + B_non_converted)

In [49]:
weight = (90+80) / (90+80+5910+3920)
weight

0.017

In [50]:
#Lalu bikin tabel baru dengan p_hat sebagai weighted conversion rate
data3 = data2.copy()
data3['converted'] = data3['number_of_sessions'] * p_hat
data3['non_converted'] = data3['number_of_sessions'] - data3['converted']
data3

,number_of_sessions,converted,non_converted
A,6000,102.0,5898.0
B,4000,68.0,3932.0


In [51]:
#kolom yang akan dipake adalah converted, dan non_converted dari data_2 (O) dan data_3 (T)
T = np.array([102, 68, 5898, 3932])
O = np.array([90, 80, 5910, 3920])

D = np.sum(np.square(T-O)/T)

pvalue = chi2.sf(D, df=1)

print(f'Nilai Distance adalag {D}')
print(f'Nilai dari p-value adalah {pvalue}')

Nilai Distance adalag 3.590449404583807
Nilai dari p-value adalah 0.05811252579106675


### Conclusions

- Karena nilai p-value > alpha maka dapat ditarik kesimpulan bahwa Fail to Reject H0 bahwa desain A dan B memiliki conversion rate yang sama!

## Final Conclusion

- Setelah melakukan A/B testing akhirnya rangga mendapatkan kesimpulan yang akan dia laporkan ke bosnya yaitu:
    1. Jika dilihat untuk rata-rata user menghabiskan waktu di website per session didapatkan bahwa desain B lebih bisa membuat user untuk bertahan lebih lama di website.
    2. Jika dilihat dari conversion rate didapatkan kesimpulan bahwa kedua desain memberikan nilai conversion rate yang sama!
    3. Saran saya adalah gunakan desain B karena Desain B mampu membuat user untuk bertahan lebih lama di website sehingga kemungkinan untuk melakukan transaksi akan tinggi.